In [18]:
import os
import cv2
import numpy as np

DATASET_DIR = './brain_tumor_dataset'
global_min_pixel = 255
global_max_pixel = 0
global_min_height_res = 99999
global_min_width_res = 99999
global_max_height_res = 0
global_max_width_res = 0
lower_bound_width = 0
upper_bound_width = 0
lower_bound_height = 0
upper_bound_height = 0

for root, dirs, files in os.walk(DATASET_DIR):
    for file_name in files:
        if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):          
            image_path = os.path.join(root, file_name)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            
            if image is not None:
                h, w = image.shape
                global_min = min(global_min_pixel, np.min(image))
                global_max = max(global_max_pixel, np.max(image))
                global_max_height_res = max(global_max_height_res, np.max(h))
                global_max_width_res = max(global_max_width_res, np.max(w))
                global_min_height_res = min(global_min_height_res, np.min(h))
                global_min_width_res = min(global_min_width_res, np.min(w))
                if w < 224:
                    lower_bound_width+=1
                else:
                    upper_bound_width+=1
                if h < 224:
                    lower_bound_height+=1
                else:
                    upper_bound_height+=1
            else:
                print(f"Warning: Could not read image at {image_path}")
print(f"Overall Minimum Pixel Intensity: {global_min}")
print(f"Overall Maximum Pixel Intensity: {global_max}")
print(f"Overall Maximum Height: {global_max_height_res}")
print(f"Overall Minimum Height: {global_min_height_res}")
print(f"Overall Maximum Width: {global_max_width_res}")
print(f"Overall Minimum Width: {global_min_width_res}")
print(f"Height Bounds: {lower_bound_height} : {upper_bound_height}")
print(f"Width Bounds: {lower_bound_width} : {upper_bound_width}")

Overall Minimum Pixel Intensity: 0
Overall Maximum Pixel Intensity: 255
Overall Maximum Height: 1427
Overall Minimum Height: 168
Overall Maximum Width: 1920
Overall Minimum Width: 150
Height Bounds: 26 : 227
Width Bounds: 50 : 203


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
DATASET_DIR = './brain_tumor_data/'

train_datagen = ImageDataGenerator(
    rescale=1./255,           # Normalize pixel values from 0-255 -> 0-1
    horizontal_flip=True,     # Randomly flip images horizontally to help with overfitting
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    directory=DATASET_DIR + 'train',
    target_size=IMAGE_SIZE,          # Resize all images to 224x224
    batch_size=BATCH_SIZE,
    class_mode='binary'              # 0/1 classification
)

validation_generator = validation_datagen.flow_from_directory(
    directory=DATASET_DIR + 'validation',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)